C++ presentation of 'Templet application' interface and model
=============================================================

```
 -----------------------------------------------------------------------
             Commands           |                Roles
 -----------------------------------------------------------------------
  REST commands |  CLI commands | RESOURCE | OBSERVER | CLIENT | SERVER
 ---------------|---------------|---------------------------------------
 write_event    |  event        |    +     |          |        |
 read_events    |  log          |    +     |    +     |        |
 reply_on_query |  reply        |          |          |        |   +
 write_query    |  query        |          |          |   +    |
 read_answer    |  answer       |          |    +     |   +    |

SESSION(NAME,RESOURCE,OBSERVER,CLIENT,SERVER)
EVENT(ORDINAL,TAG,EXTERN,NAME,DATA)
ANSWER(ORDINAL,NAME,DATA)

write_event(SESSION,TAG,DATA)   --> ORDINAL \/ false
read_events(SESSION,ORDINAL)    --> EVENT[] \/ false
write_query(SESSION,TAG,DATA)   --> ORDINAL \/ false
reply_on_query(SESSION,ORDINAL,DATA) --> true \/ false
read_answer(SESSION,ORDINAL)    --> DATA \/ false
```

In [1]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

public: // service interface    

typedef string   NAME;
typedef unsigned ORDINAL;
typedef unsigned TAG;
typedef bool     EXTERN;

#define ASSERT(expr)  if(!(expr))return false;
    
struct SESSION{
    NAME name;    // who is the session owner?
    bool RESOURCE;// what access does the session owner have?
    bool OBSERVER;
    bool CLIENT;
    bool SERVER;
}; 

struct DATA{ long size; void* data; }; //abstract binary data

struct EVENT{
    ORDINAL ord;
    TAG     tag;
    EXTERN  ext;  // ext==true used for client queries
    DATA    data; // if ext==true, data may include query GUID
    NAME    name;
};

struct ANSWER{
    ORDINAL ord;  // query ordinal number
    NAME    name; // who can read the answer?
    DATA    data;
};

/*-1-*/
bool write_event(SESSION s,TAG t,DATA d,ORDINAL& ord){
   
    ASSERT(s.RESOURCE);

    // EVENT.ord is a key autoincrement field
    EVENT ev;   
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.ext = false; // regular (internal) event
    ev.data = d; ev.name = s.name;
    events.push_back(ev);
    
    return true;
}
    
/*-2-*/              
bool read_events(SESSION s,ORDINAL ord,list<EVENT>& evs){
    
    ASSERT(s.RESOURCE || s.OBSERVER);

    evs.clear();
    
    for(EVENT ev:events){
        if(ev.ord>=ord) evs.push_back(ev);
    }   
    return true;
}

/*-3-*/
bool write_query(SESSION s,TAG t,DATA d){
    
    ASSERT(s.CLIENT);
    
    EVENT ev;
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.ext = true; // external event 
    ev.data = d; ev.name = s.name;   
    events.push_back(ev);
    
    return true;
}

/*-4-*/
bool reply_on_query(SESSION s,ORDINAL ord,DATA d){

    ASSERT(s.SERVER);

    // ANSWER.ord is a key field
    for(ANSWER ans:answers){
        if(ans.ord==ord) return false;
    }

    for(EVENT ev:events){
        if(ev.ord==ord && ev.ext){
            ANSWER ans;
            ans.ord = ord; ans.name = ev.name; ans.data = ev.data;
            answers.push_back(ans);
            return true;
        }
    }  
    return true;
}    

/*-5-*/            
bool read_answer(SESSION s,ORDINAL ord,DATA& d){
   
    ASSERT(s.CLIENT || s.OBSERVER);

    for(ANSWER ans:answers){
        if(ans.ord == ord){
            ASSERT(s.CLIENT ? s.name==ans.name : s.OBSERVER);
            d = ans.data;
            return true;
        }
    }   
    return false;
}

private: // service state
    
list<EVENT>  events;
list<ANSWER> answers;

};

```
Интерфейс параметров командной строки
=====================================

Идея: возможно реализовать сервис синхронизации глобального состояния на базе готовых систем удаленного запуска задач (Everest, HTCondor, Dirac и т.д.) для быстрого прототипирования и использования в качестве эталонной реализации для полнофункционального сервиса, а затем CLI-интерфейса полнофункционального сервиса.
Для этого реализуется консольное приложение с описанным ниже интерфейсом.

Параметры
=========
(--base | -b) <base> база данных состояния приложения или endpoint приложения
(--jwt  | -j) <JWT> токен с описанием прав доступа для агента с заданным именем
(--tag  | -t) <num> тэг события
(--ordinal | -о) <num> порядковый номер события

Форматы файлов и stdout
=======================
<file>     формат определяется в приложении
<base>     SQLite файл базы данных, хранящей состояние приложения, или endpoint для подключения сереру приложения
<logfile>  запись фрагмента последовательных событий обновления состояния, внутренний формат

Команды
=======
(--event | -e) <file>  запись события, параметры: base, tag, в stdout - порядковый номер события
(--log   | -l) <logfile> выборка из лога событий, начиная с заданного ordinal, параметры: base, ordinal
(--reply | -r) <file>  ответ на запрос с указанным номером ordinal, параметры: base, ordinal
(--query | -q) <file>  файл с данными запроса, параметры: base, tag, в stdout - порядковый номер события-запроса
(--answer| -a) <file>  файл с ответом на запрос, параметры: base, ordinal
```